In [ ]:
# === Imports ===
import pandas as pd
import json
import joblib
import spacy
import re

# === 1. Função auxiliar para carregar JSON ===
def carregar_json(caminho):
    with open(caminho, 'r', encoding='utf-8') as f:
        return json.load(f)

# === 2. Carregar os arquivos JSON ===
jobs = carregar_json('vagas.json')
prospects = carregar_json('prospects.json')
applicants = carregar_json('applicants.json')

# === 3. Transformar em DataFrames ===
df_jobs = pd.DataFrame.from_dict(jobs, orient='index').reset_index().rename(columns={'index': 'job_id'})
df_applicants = pd.DataFrame.from_dict(applicants, orient='index').reset_index().rename(columns={'index': 'applicant_id'})

lista_prospects = []
for job_id, value in prospects.items():
    if isinstance(value, list):
        for candidato in value:
            if isinstance(candidato, dict):
                lista_prospects.append({'job_id': job_id, **candidato})
    elif isinstance(value, dict) and 'prospects' in value:
        for candidato in value['prospects']:
            if isinstance(candidato, dict):
                lista_prospects.append({'job_id': job_id, **candidato})

df_merge = pd.DataFrame(lista_prospects)
df_merge = df_merge.merge(df_applicants, left_on='codigo', right_on='applicant_id', how='left')
df_merge = df_merge.merge(df_jobs, on='job_id', how='left')

# === 4. Pré-processamento ===
nlp = spacy.load('pt_core_news_sm')

def extrair_entidades(texto):
    doc = nlp(texto)
    return [(ent.text, ent.label_) for ent in doc.ents]

colunas_texto = ['formacao_e_idiomas', 'cargo_atual', 'perfil_vaga', 'beneficios',
                 'informacoes_profissionais', 'infos_basicas']
df_merge[colunas_texto] = df_merge[colunas_texto].fillna('')
df_merge['texto_geral'] = df_merge[colunas_texto].astype(str).agg(' '.join, axis=1)
df_merge['entidades'] = df_merge['texto_geral'].apply(extrair_entidades)

# === 5. Criar colunas binárias com os termos usados no modelo treinado ===
top_entidades = {
    'PER': ['ensino superior completo', 'pleno', 'ensino médio completo','srta', 'inicio', 'ensino', 'conhecimento','completo', 'budgeted rate', 'dra'],
    'ORG': ['p1', 'pj', 'p3 - advanced', 'sd', 'iso','pp', 'microsoft', 'ibm', 'itil', 'pmo'],
    'LOC': ['brasil', 'são paulo', 'avançado', 'intermediário', 'fechado','experiência', 'rio de janeiro', 'sp', 'minas gerais', 'fortaleza'],
    'MISC': ['básico', 'sap', 'ti - projetos-', 'r$', 'gestão','notebook', 'sql', 'alocação de recursos de ti-','oracle', 'java']
}

for tipo, termos in top_entidades.items():
    for termo in termos:
        col = f"{tipo}_{termo}".replace(" ", "_").lower()
        df_merge[col] = df_merge['entidades'].apply(lambda lista: int((termo, tipo) in lista))

# === 6. Carregar modelo treinado ===
modelo = joblib.load('modelo_rf_final.pkl')
threshold = 0.75

# === 7. Garantir que todas as colunas do modelo existam no df_merge ===
for col in modelo.feature_names_in_:
    if col not in df_merge.columns:
        df_merge[col] = 0  # preencher com 0 caso não exista

# === 8. Previsão ===
X_novo = df_merge[modelo.feature_names_in_]
probs = modelo.predict_proba(X_novo)[:, 1]
df_merge['prob_contratacao'] = probs
df_merge['contratado_predito'] = (probs >= threshold).astype(int)

# === 9. Salvar resultado ===
df_merge[['codigo', 'job_id', 'prob_contratacao', 'contratado_predito']].to_csv('predicoes.csv', index=False)
print(df_merge[['codigo', 'job_id', 'prob_contratacao', 'contratado_predito']].head())